# Elliptic Curves: Starter Challenges

## Background

> The flag is the name we give groups with a commutative operation.

These are Abelian groups. The flag is `crypto{Abelian}`.

## Starter 1: Point Negation

> `E: Y^2 = X^3 + 497 X + 1768, p: 9739`

> `P(8045,6936)`

> Find the point `Q(x, y)` such that `P + Q = O`.

To negate a point, we need to find some point which we can connect to P with a line, where the line then does not hit the curve. We do this by creating a vertical line, so the point -P is directly below P on the curve. This implies we should just need to flip the y coordinate, and leave x unchanged.

Solution: `Q = - P = (8045, 9739 - 6936) = (8045, 2803)`.

That makes the flag `crypto{8045, 2803}`.

## Starter 2: Point Addition

> `E: Y^2 = X^3 + 497 X + 1768, p: 9739`

> Using the above curve, and the points `P = (493, 5564), Q = (1539, 4742), R = (4403,5202)`, find the point `S(x,y) = P + P + Q + R` by implementing the above algorithm.

Let me use this helper class:

In [31]:
class EllipticCurve:
    def __init__(self, a, b, p):
        self.a = a 
        self.b = b 
        self.p = p 

class Point:
    def __init__(self, x=False, y=False):
        self.x = x 
        self.y = y
        # Is this a point at infinity?
        self.o = not (type(x) == int and type(y) == int)
 

    def __eq__(self, q): 
        return (self.x == q.x and self.y == q.y) or (self.o and q.o)

    def __ne__(self, q): 
        return not (self == q)

This is the algorithm given in the theory:

In [32]:
def point_add(p, q, ec):
    if p.o:
        return q
    if q.o:
        return p
    if p.x == q.x and p.y == -q.y:
        return Point()

    if p != q:
        x_inv = pow(q.x - p.x, -1, ec.p)
        lam = ((q.y - p.y) * x_inv) % ec.p
    else:
        y_inv = pow(2 * p.y, -1, ec.p)
        lam = (((3*(p.x**2)) + ec.a) * y_inv) % ec.p

    res_x = ((lam**2) - p.x - q.x) % ec.p
    res_y = ((lam * (p.x - res_x)) - p.y) % ec.p
    
    return Point(res_x, res_y)

Which we can now use on the given curve and the given points:

In [33]:
ec = EllipticCurve(497, 1768, 9739)

p = Point(493, 5564)
q = Point(1539, 4742)
r = Point(4403, 5202)

result = point_add(point_add(point_add(p, p, ec), q, ec), r, ec)

print("crypto{" + str(result.x) + "," + str(result.y) + "}")

crypto{4215,2162}


## Starter 3: Scalar Multiplication

> Using the above curve, and the points P = (2339, 2213), find the point Q(x,y) = 7863 P by implementing the above algorithm.



In [34]:
def is_odd(n):
    return n % 2 == 1

def scalar_mult(p, n, ec):
    q = Point(p.x, p.y)
    r = Point()
    while n > 0:
        if is_odd(n):
            r = point_add(r, q, ec)
        q = point_add(q, q, ec)
        n //= 2
    return r

We're given a test case:

In [35]:
test_point = Point(5323, 5438)
test_result = scalar_mult(test_point, 1337, ec)
assert test_result.x == 1089
assert test_result.y == 6931

Yay! The solution:

In [36]:
p = Point(2339, 2213)
result = scalar_mult(p, 7863, ec)
print("crypto{" + str(result.x) + "," + str(result.y) + "}")

crypto{9467,2742}


## Starter 4: Curves and Logs

> Calculate the shared secret after Alice sends you QA = (815, 3190), with your secret integer nB = 1829.


> Generate a key by calculating the SHA1 hash of the x coordinate (take the decimal representation of the coordinate and cast it to a string). The flag is the hexdigest you find.


In [37]:
from hashlib import sha1

ec = EllipticCurve(497, 1768, 9739)

qa = Point(815, 3190)
shared_secret = scalar_mult(qa, 1829, ec)

hash = sha1(str(shared_secret.x).encode()).hexdigest()

print("crypto{" + hash + "}")

crypto{80e5212754a824d3a4aed185ace4f9cac0f908bf}


## Starter 5: Efficient Exchange

> Calculate the shared secret after Alice sends you q_x = 4726, with your secret integer nB = 6534.

> Use the decrypt.py file to decode the flag

This file for decryption is given. Using q_x and the Weierstrass equation, we can calculate x and y.

In [38]:
given_x = 4726
y_squared =  ((given_x ** 3) + (497 * given_x) + 1768) % 9739
given_y = pow(5507, (9740 // 4), 9739)


qa = Point(given_x, given_y)
shared_secret = scalar_mult(qa, 6534, ec)

In [39]:
given_message = {
    'iv': 'cd9da9f1c60925922377ea952afc212c', 
    'encrypted_flag': 'febcbe3a3414a730b125931dccf912d2239f3e969c4334d95ed0ec86f6449ad8'
}

In [40]:
!pip install pycryptodome

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [42]:
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
import hashlib


def is_pkcs7_padded(message):
    padding = message[-message[-1]:]
    return all(padding[i] == len(padding) for i in range(0, len(padding)))


def decrypt_flag(shared_secret: int, iv: str, ciphertext: str):
    # Derive AES key from shared secret
    sha1 = hashlib.sha1()
    sha1.update(str(shared_secret).encode('ascii'))
    key = sha1.digest()[:16]
    # Decrypt flag
    ciphertext = bytes.fromhex(ciphertext)
    iv = bytes.fromhex(iv)
    cipher = AES.new(key, AES.MODE_CBC, iv)
    plaintext = cipher.decrypt(ciphertext)

    if is_pkcs7_padded(plaintext):
        return unpad(plaintext, 16).decode('ascii')
    else:
        return plaintext.decode('ascii')


print(decrypt_flag(shared_secret.x, given_message['iv'], given_message['encrypted_flag']))

crypto{3ff1c1ent_k3y_3xch4ng3}
